1. **Based on:** https://www.youtube.com/watch?v=fyJB1t0o0ms
1. **Dataset:** https://www.kaggle.com/datasets/nilesh14k/highway-car-video

In [ ]:
!pip install -q ultralytics
!pip install -q supervision==0.11.1

In [ ]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
import easyocr
import string
import cv2
import os

In [ ]:
model = YOLO("yolov8n.pt")
license_plate_detector = YOLO("/kaggle/input/license-plate-detector/license_plate_detector.pt")

In [ ]:
reader = easyocr.Reader(['en'])
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

def license_complies_format(text):
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False


def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)
    
    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')
        if license_complies_format(text):
            return format_license(text), score

    return None, None


def get_vehicle(license_plate_bbox, vehicle_tracks):
    x1, y1, x2, y2 = license_plate_bbox
    
    found_vehicle = None
    for vehicle_idx in range(len(vehicle_tracks)):
        x_vehicle1, y_vehicle1, x_vehicle2, y_vehicle2 = vehicle_tracks.xyxy[vehicle_idx]
        if x1 >= x_vehicle1 and y1 >= y_vehicle1 and x2 <= x_vehicle2 and y2 <= y_vehicle2:
            found_vehicle = vehicle_tracks.xyxy[vehicle_idx]
            break

    if found_vehicle is not None:
        vehicle_id = vehicle_tracks.tracker_id[vehicle_idx]
        vehicle_class_id = vehicle_tracks.class_id[vehicle_idx]
        return found_vehicle, vehicle_id, vehicle_class_id
    
    return [-1, -1, -1, -1], -1, -1

def get_max_score(partial_results):
    results = {}
    for vehicles in partial_results.values():
        for vehicle_id in vehicles.keys():
            if vehicle_id not in results.keys():
                results[vehicle_id] = vehicles[vehicle_id]
            else:
                if results[vehicle_id]['license_plate']['text_score'] < vehicles[vehicle_id]['license_plate']['text_score']:
                    results[vehicle_id] = vehicles[vehicle_id]
    return results

def write_results(results, file):
    if os.path.isfile(file):
        os.remove(file)
    with open(file, 'a+') as f:
        f.write('id;license_plate_;license_plate_score' + '\n')
        
        for vehicle_id in results.keys():
            f.write(f'{vehicle_id};{results[vehicle_id]["license_plate"]["text"]};{results[vehicle_id]["license_plate"]["text_score"]:.2f}' + '\n')
        

In [ ]:
def track(video, save = False, output = ''):
    results = {}
    partial_results = {}
    vehicles_id = [2, 3, 5, 7]

    if save:
        cap = cv2.VideoCapture(video)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = cap.get(cv2.CAP_PROP_FPS)
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        video_writer = cv2.VideoWriter(output, fourcc, fps, (width, height))
        
    for frame_idx, result in enumerate(model.track(source = video, show = False, stream = True)):
        partial_results[frame_idx] = {}
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        if result.boxes.id is not None:
            detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)

            detections = detections[np.isin(detections.class_id, vehicles_id)]

        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates:
            class_id = license_plate.boxes.cls
            confidence = license_plate.boxes.conf.cpu().item()
            x1, y1, x2, y2 = license_plate.boxes.xyxy[0].cpu().numpy()
            vehicle_tracker_loc, vehicle_tracker_id, vehicle_tracker_class_id = get_vehicle(license_plate.boxes.xyxy[0], detections)
            
            if vehicle_tracker_id == -1:
                continue

            license_plate_crop = frame[int(y1):int(y2), int(x1):int(x2),:]
            
            license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
            
            _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)
            
            license_plate_text, license_plate_confidence = read_license_plate(license_plate_crop_thresh)
            
            if license_plate_text is not None:
                partial_results[frame_idx][vehicle_tracker_id] = {'vehicle': {'bbox': vehicle_tracker_loc,
                                                                      'tracker_id': vehicle_tracker_id},
                                                             'license_plate': {
                                                                 'bbox': [x1, y1, x2, y2],
                                                                 'text': license_plate_text,
                                                                 'bbox_score': confidence,
                                                                 'text_score': license_plate_confidence}
                                                      }
                if save:
                    frame = cv2.rectangle(frame, (int(vehicle_tracker_loc[0]),int(vehicle_tracker_loc[1])), (int(vehicle_tracker_loc[2]), int(vehicle_tracker_loc[3])), (255, 0, 0), 2)
                    frame = cv2.rectangle(frame, (int(x1),int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                    frame = cv2.putText(frame, f'class: {model.model.names[vehicle_tracker_class_id]} id: {vehicle_tracker_id} license_plate: {license_plate_text}',(int(vehicle_tracker_loc[0]), int(vehicle_tracker_loc[1]) - 1), cv2.FONT_HERSHEY_SIMPLEX, 1,color = (0, 255, 255))
                    
        if save:
            video_writer.write(frame)

    if save:
        video_writer.release()
    
    results = get_max_score(partial_results)
    return results

In [ ]:
video = '/kaggle/input/highway-car-video/Pexels Videos 2103099.mp4'
file = '/kaggle/working/results.csv'

results = track(video, save = True, output = '/kaggle/working/output.mp4')
write_results(results, file)